## Построение сценического графа по текстовому описанию сцены

### Итоговая валидация модели

архитектура модели:



In [99]:
import transformers
import datasets
import huggingface_hub
import torch

import json
import torch
import os
import sys
import warnings
import random
import glob

import numpy as np

from collections import Counter, defaultdict
from typing import List, Dict
from pathlib import Path
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import TrainerCallback

from peft import LoraConfig, get_peft_model, TaskType, PeftConfig,PeftModel
from tqdm import tqdm

# отключаем их все чтобы картинку не портили
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_DIR = Path("../dataset/dataset_validation_spacial").expanduser()
MODEL_NAME = "sberbank-ai/ruT5-base"

lib_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(lib_path)

from library.final_metrics import evaluate_obj_attr_metrics, evaluate_ged_score
from library.utils import json_to_pseudo_text, pseudo_text_to_json

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Путь к папке с датасетом
DATASET_DIR = "../dataset/dataset_validation_spacial"
VAL_SPLIT = 1.0

In [100]:
import warnings
warnings.filterwarnings("ignore")

from transformers.utils import logging
logging.set_verbosity_error()

### Параметры для модели, определяющей объекты и признаки

In [101]:
MODEL_DIR_OBJ = "../models/T5ru_PsC_lora_outputs"  

INPUT_SEQ_LENGTH_OBJ = 1100
OUTPUT_SEQ_LENGTH_OBJ = 512

NUM_BEAMS_OBJ = 8

PROMPT_OBJ = """
Ты должен проанализировать описание сцены и вернуть ответ в специальном псевдоформате.

Твоя задача:
- Найди все объекты, упомянутые в описании, и их признаки.
- Верни результат строго в псевдоформате — одной строкой.

Формат:
объект1 (признак1 признак2) объект2 () объект3 (признак)

Требования:
- Каждый объект указывается один раз.
- Признаки пишутся через пробел внутри круглых скобок.
- Если признаки отсутствуют, используй пустые скобки ().
- Не добавляй объектов или признаков, которых нет в описании.
- В ответе не должно быть никаких пояснений, комментариев или заголовков — только одна строка с результатом.

Примеры:

Описание: Маленький красный стол стоит у окна.
Ответ:
стол (маленький красный) окно ()

Описание: {description}

Ответ:
"""


### Выделение объектов и признаков

In [102]:
def make_objects_attrs(description):
    """
    по текстовому описанию генерирует список объектов и атрибутов для сцены
    """

    # Загрузка модели и токенизатора
    config = PeftConfig.from_pretrained(MODEL_DIR_OBJ)
    base_model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base_model, MODEL_DIR_OBJ)
    model = model.to(DEVICE)
    model.eval()
    
    #print(model)
    
    tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)
    
    prompt = PROMPT_OBJ.format(description=description)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=INPUT_SEQ_LENGTH_OBJ
    ).to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=OUTPUT_SEQ_LENGTH_OBJ,
            num_beams=NUM_BEAMS_OBJ, # попробовать меньше
            #temperature=TEMPERATURE, # параметризовать
            early_stopping=True
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    try:
        parsed_json = pseudo_text_to_json(output_text)
    except Exception as e:
        print(f"Ошибка парсинга JSON: {e}")
        print("Сырые данные:", output_text)
        parsed_json = None

    return parsed_json


In [103]:
description = "На столе стояла синяя лампа а рядом с ней жёлтый учебник"
make_objects_attrs(description)



[{'учебник': ['жёлтый']}, {'лампа': ['синяя']}]

### Параметры для модели, выделяющей пространственные признаки

In [104]:
MODEL_DIR_SP = "../models/T5ru_spacial_lora_outputs" 

MAX_INPUT_LENGTH_SP = 512 
MAX_OUTPUT_LENGTH_SP = 32 

NUM_BEAMS_SP = 8

PROMPT_SP = """
Определи пространственную связь между объектами '{obj1}' и '{obj2}'
в следующем описании сцены: {description}"""

In [105]:
def make_spatial_relations(description, obj1, obj2):
    """
    по заданному текстовому описанию 
    """
    # Загрузка модели и токенизатора
    config = PeftConfig.from_pretrained(MODEL_DIR_SP)
    base_model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base_model, MODEL_DIR_SP)
    model = model.to(DEVICE)
    model.eval()
    
    #print(model)
    tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)
    
    prompt = PROMPT_SP.format(obj1=obj1, obj2=obj2, description=description)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=MAX_INPUT_LENGTH_SP
    ).to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=MAX_OUTPUT_LENGTH_SP,
            num_beams=NUM_BEAMS_SP,
            early_stopping=True
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [106]:
description = "кот рядом с полкой"
make_spatial_relations(description, "полка", "кот")


'нет связи'

In [107]:
def extract_spatial_triplets(description, objects_with_attrs):
    object_names = [list(obj.keys())[0] for obj in objects_with_attrs]  # ['учебник', 'лампа']
    
    triplets = []

    for i in range(len(object_names)):
        for j in range(len(object_names)):
            if i == j:
                continue  # пропустить пары с одинаковыми объектами
            obj1 = object_names[i]
            obj2 = object_names[j]
            relation = make_spatial_relations(description, obj1, obj2)
            if relation != "нет связи":
                triplets.append([obj1, relation, obj2])
    
    return triplets

In [108]:
description = "На столе лежит жёлтый учебник. Слева от него стоит синяя лампа."

triplets = extract_spatial_triplets(description, make_objects_attrs(description))
print(triplets)

[['лампа', 'слева от', 'учебник']]


## Сборка финальной модели



In [109]:
def T5ru_model(description, location="неизвестно"):
    # Извлечение предсказаний
    obj_attr = make_objects_attrs(description)
    relations = extract_spatial_triplets(description, obj_attr)      
    
    scene = dict()
    scene["location"] = location
    scene["objects"] = obj_attr
    scene["relations"] = relations
    
    return {"scene": scene}

In [110]:
description = "На столе лежит жёлтый учебник. Слева от него стоит синяя лампа."
T5ru_model(description)

{'scene': {'location': 'неизвестно',
  'objects': [{'учебник': ['жёлтый']}, {'лампа': ['синяя']}],
  'relations': [['лампа', 'слева от', 'учебник']]}}

### Большой финальный тест на валидационной выборке

In [111]:
# Загружаем все jsonl-файлы из датасета
def load_dataset(path: str) -> List[Dict]:
    dataset = []
    for filename in glob.glob(os.path.join(path, "*.jsonl")):
        with open(filename, "r", encoding="utf-8") as f:
            for line in f:
                dataset.append(json.loads(line))
    return dataset

# Получаем .05 данных
def sample_validation_split(dataset: List[Dict], fraction: float = 0.05) -> List[Dict]:
    sample_size = max(1, int(len(dataset) * fraction))
    return random.sample(dataset, sample_size)

# Обработка + сбор метрик
def evaluate_on_validation_set(dataset: List[Dict]) -> Dict[str, float]:
    metrics_accumulator = defaultdict(list)
    
    for item in tqdm(dataset, ncols=80):
        src_text = item["description"]        
        if not src_text:
            print("Пустое или отсутствующее поле 'description' в элементе:")
            print(item)
            continue
        
        pred = T5ru_model(src_text)
        label = {"scene": item["scene"]}
        
        # Оценка для базовых метрик
        metrics = evaluate_obj_attr_metrics(pred, label)
        for k, v in metrics.items():
            metrics_accumulator[k].append(v)

        # Оценка с точки зрения графа целиком
        metrics = evaluate_ged_score(pred, label)
        for k, v in metrics.items():
            metrics_accumulator[k].append(v)
            
            
    # Усреднение по всем примерам
    return {k: round(sum(vs) / len(vs), 4) for k, vs in metrics_accumulator.items()}


In [112]:
all_data = load_dataset(DATASET_DIR)
val_data = sample_validation_split(all_data, VAL_SPLIT)
final_metrics = evaluate_on_validation_set(val_data)

print("Validation results on", len(all_data), "samples:")
for k, v in final_metrics.items():
    print(f"{k}: {v}")

100%|███████████████████████████████████████| 250/250 [2:27:22<00:00, 35.37s/it]

Validation results on 250 samples:
f1_objects: 0.9826
f1_attributes_macro: 0.7367
f1_attributes_weighted: 0.9296
f1_global_obj_attr_pairs: 1.0
f1_combined_simple: 0.8597
f1_combined_weighted: 0.9593
GED_score: 0.564
